# Deep Q-Learning - Lunar Lander

Dans le cadre de cette mission, vous formerez un agent à l'atterrissage d'un module lunaire en toute sécurité sur une plate-forme d'atterrissage à la surface de la lune.


# Outline
- [ 1 - Import Packages <img align="Right" src="./images/lunar_lander.gif" width = 60% >](#1)
- [ 2 - Hyperparameters](#2)
- [ 3 - The Lunar Lander Environment](#3)
  - [ 3.1 Action Space](#3.1)
  - [ 3.2 Observation Space](#3.2)
  - [ 3.3 Rewards](#3.3)
  - [ 3.4 Episode Termination](#3.4)
- [ 4 - Load the Environment](#4)
- [ 5 - Interacting with the Gym Environment](#5)
    - [ 5.1 Exploring the Environment's Dynamics](#5.1)
- [ 6 - Deep Q-Learning](#6)
  - [ 6.1 Target Network](#6.1)
    - [ Exercise 1](#ex01)
  - [ 6.2 Experience Replay](#6.2)
- [ 7 - Deep Q-Learning Algorithm with Experience Replay](#7)
  - [ Exercise 2](#ex02)
- [ 8 - Update the Network Weights](#8)
- [ 9 - Train the Agent](#9)
- [ 10 - See the Trained Agent In Action](#10)
- [ 11 - Congratulations!](#11)
- [ 12 - References](#12)


<a name="1"></a>
## 1 - Importer des paquets

Nous allons utiliser les paquets suivants :
- `numpy` est un package pour le calcul scientifique en python.
- `deque` sera notre structure de données pour notre mémoire tampon.
- `namedtuple` sera utilisé pour stocker les tuples d'expérience.
- La boîte à outils `gym` est une collection d'environnements qui peuvent être utilisés pour tester les algorithmes d'apprentissage par renforcement. Nous devons noter que dans ce carnet nous utilisons la version `gym` `0.24.0`.
- `PIL.Image` et `pyvirtualdisplay` sont nécessaires pour rendre l'environnement Lunar Lander.
- Nous allons utiliser plusieurs modules du framework `tensorflow.keras` pour construire des modèles d'apprentissage profond.
- `utils` est un module qui contient des fonctions d'aide pour ce travail. Vous n'avez pas besoin de modifier le code dans ce fichier.

Exécutez la cellule ci-dessous pour importer tous les paquets nécessaires.

In [6]:
import time
from collections import deque, namedtuple
import gym
import numpy as np
import PIL.Image
import tensorflow as tf
import utils

from pyvirtualdisplay import Display
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import MSE
from tensorflow.keras.optimizers import Adam

In [7]:
# Set up a virtual display to render the Lunar Lander environment.
Display(visible=0, size=(840, 480)).start();

# Set the random seed for TensorFlow
tf.random.set_seed(utils.SEED)

<a name="2"></a>
## 2 - Hyperparamètres

Exécutez la cellule ci-dessous pour définir les hyperparamètres.

In [8]:
MEMORY_SIZE = 100_000     # size of memory buffer
GAMMA = 0.995             # discount factor
ALPHA = 1e-3              # learning rate  
NUM_STEPS_FOR_UPDATE = 4  # perform a learning update every C time steps

<a name="3"></a>
## 3 - L'environnement de l'atterrisseur lunaire

Dans ce cahier, nous utiliserons [OpenAI's Gym Library](https://www.gymlibrary.ml/). La bibliothèque Gym fournit une grande variété d'environnements pour l'apprentissage par renforcement. Pour simplifier, un environnement représente un problème ou une tâche à résoudre. Dans ce cahier, nous allons essayer de résoudre l'environnement Lunar Lander en utilisant l'apprentissage par renforcement.

L'objectif de l'environnement Lunar Lander est de faire atterrir l'atterrisseur lunaire en toute sécurité sur la plate-forme d'atterrissage à la surface de la lune. La plateforme d'atterrissage est désignée par deux mâts et se trouve toujours aux coordonnées `(0,0)` mais l'atterrisseur est également autorisé à atterrir en dehors de la plateforme d'atterrissage. L'atterrisseur démarre au centre supérieur de l'environnement avec une force initiale aléatoire appliquée à son centre de masse et dispose d'un carburant infini. L'environnement est considéré comme résolu si vous obtenez `200` points. 

<br>
<br>
<figure>
  <img src = "images/lunar_lander.gif" width = 40%>
      <figcaption style = "text-align: center; font-style: italic">Fig 1. Lunar Lander Environment.</figcaption>
</figure>



<a name="3.1"></a>
### 3.1 Espace d'action

L'agent dispose de quatre actions discrètes :

* Ne rien faire.
* Mettre le moteur droit en marche.
* Mettre le moteur principal en marche.
* Mettre le moteur gauche en marche.

Chaque action a une valeur numérique correspondante :

```python
Do nothing = 0
Fire right engine = 1
Fire main engine = 2
Fire left engine = 3
```

<a name="3.2"></a>
### 3.2 Espace d'observation

L'espace d'observation de l'agent consiste en un vecteur d'état avec 8 variables :

* Ses coordonnées $(x,y)$. La plateforme d'atterrissage est toujours aux coordonnées $(0,0)$.
* Ses vitesses linéaires $(\dot x,\dot y)$.
* Son angle $\theta$.
* Sa vitesse angulaire $\dot \theta$.
* Deux booléens, $l$ et $r$, qui représentent si chaque jambe est en contact avec le sol ou non.

<a name="3.3"></a>
### 3.3 Récompenses

L'environnement Lunar Lander dispose du système de récompense suivant :

* L'atterrissage sur la plate-forme d'atterrissage et l'immobilisation de l'engin rapportent environ 100 à 140 points.
* Si l'atterrisseur s'éloigne de la plate-forme d'atterrissage, il perd des points. 
* Si l'atterrisseur s'écrase, il reçoit -100 points.
* Si l'atterrisseur s'immobilise, il reçoit +100 points.
* Chaque jambe en contact avec le sol est récompensée par +10 points.
* L'allumage du moteur principal rapporte -0,3 point par image.
* L'allumage du moteur latéral rapporte -0,03 point par image.

<a name="3.4"></a>
### 3.4 Fin de l'épisode

Un épisode se termine (c'est-à-dire que l'environnement entre dans un état terminal) si :

* l'atterrisseur lunaire s'écrase (c'est-à-dire si le corps de l'atterrisseur lunaire entre en contact avec la surface de la lune).

* La coordonnée $x$ de l'atterrisseur est supérieure à 1.

Vous pouvez consulter la [documentation Open AI Gym] (https://www.gymlibrary.ml/environments/box2d/lunar_lander/) pour une description complète de l'environnement. 

<a name="4"></a>
## 4 - Charger l'environnement

Nous commençons par charger l'environnement `LunarLander-v2` depuis la bibliothèque `gym` en utilisant la méthode `.make()`. `LunarLander-v2` est la dernière version de l'environnement Lunar Lander et vous pouvez lire l'historique de ses versions dans la [documentation Open AI Gym] (https://www.gymlibrary.ml/environments/box2d/lunar_lander/#version-history).

In [9]:
env = gym.make('LunarLander-v2')

DependencyNotInstalled: box2D is not installed, run `pip install gym[box2d]`